# Initialization

Импорт нужных библиотек

In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [3]:
items = pd.read_parquet('goodsread/items.parquet')
events = pd.read_parquet('goodsread/events.parquet')

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [4]:
#items.head()
#events.head()

Завершите код для разбиения всех событий. В качестве точки разбиения используйте 2017-08-01, то есть отнесите в тестовую часть три последних месяца.

In [5]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx] # ваш код здесь #
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates() # ваш код здесь #
# количество пользователей, которые есть и в train, и в test
common_users = users_train[users_train.isin(users_test)] # ваш код здесь #

print(len(users_train), len(users_test), len(common_users)) 

428220 123223 120858


# === Знакомство: первые персональные рекомендации

In [6]:
cold_users =  users_test[~users_test.isin(users_train)]# ваш код здесь #

print(len(cold_users)) 

2365


Завершите код, чтобы получить топ-100 наиболее популярных книг согласно условиям выше.

In [7]:
top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()
item_popularity["popularity_weighted"] = item_popularity["users"] * item_popularity["avg_rating"]

# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values(by="popularity_weighted", ascending=False) # ваш код здесь #

# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity.query('avg_rating >= 4').head(100)# ваш код здесь # 

In [8]:
top_k_pop_items.head()

,item_id,users,avg_rating,popularity_weighted
32387,18007564,20207,4.321275,87320.0
32623,18143977,19462,4.290669,83505.0
30695,16096824,16770,4.301014,72128.0
2,3,15139,4.706057,71245.0
3718,38447,14611,4.232770,61845.0


In [9]:
# добавляем информацию о книгах которые попали в топ
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], on="item_id")

with pd.option_context('display.max_rows', 100):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_weighted", "genre_and_votes"]]) 

,item_id,author,title,publication_year,users,avg_rating,popularity_weighted,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,87320.0,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,83505.0,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,72128.0,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
3,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,71245.0,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
4,38447,Margaret Atwood,The Handmaid's Tale,1998,14611,4.232770,61845.0,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
5,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,1999,13043,4.632447,60421.0,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict..."
6,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",2012,14348,4.179189,59963.0,"{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
7,17927395,Sarah J. Maas,A Court of Mist and Fury (A Court of Thorns an...,2016,12177,4.730640,57605.0,"{'Fantasy': 10186, 'Romance': 3346, 'Young Adu..."
8,18692431,"Nicola Yoon, David Yoon","Everything, Everything",2015,14121,4.071454,57493.0,"{'Young Adult': 5175, 'Romance': 3234, 'Contem..."
9,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,2004,11890,4.770143,56717.0,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."


Завершите предлагаемый код, чтобы в cold_users_events_with_recs для каждого события получить столбец avg_rating. В нём при совпадении по item_id будет значение из одноимённого столбца из top_k_pop_items, иначе — пропуск.
В cold_users_events_with_recs после выполнения завершённого кода должно быть столько же строк, сколько было до его выполнения.

In [10]:
cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items[["item_id", "avg_rating"]], on="item_id", how="left")

cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "item_id", "rating", "avg_rating"]] 

Для какой доли событий «холодных» пользователей в events_test рекомендации в top_k_pop_items совпали по книгам? Округлите ответ до сотых.

In [11]:
# Подсчитываем количество событий "холодных" пользователей в events_test
cold_events_count = len(events_test[events_test["user_id"].isin(cold_users)])

# Подсчитываем количество событий, где top_k_pop_items совпали по книгам
matching_events_count = len(cold_users_events_with_recs.dropna(subset=["avg_rating"]))

# Вычисляем долю событий, где рекомендации совпали
matching_events_ratio = matching_events_count / cold_events_count

matching_events_ratio = round(matching_events_ratio, 2) # Округляем до сотых

print("Доля событий, где рекомендации совпали по книгам:", matching_events_ratio)

Доля событий, где рекомендации совпали по книгам: 0.2


Посчитайте метрики rmse и mae для полученных рекомендаций.

In [12]:
# посчитаем метрики рекомендаций
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])# ваш код здесь #
print(round(rmse, 2), round(mae, 2)) 

0.78 0.62


In [13]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}") 

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


Оцените степень разреженности U-I-матрицы, построенной на основе events. Используйте  данные events[['user_id', 'item_id', 'rating']]. Какое значение корректно описывает результат?

In [14]:
# Подсчет количества уникальных пар пользователь-предмет
unique_pairs_count = events[['user_id', 'item_id']].drop_duplicates().shape[0]

# Подсчет количества уникальных предметов
unique_items_count = events['item_id'].nunique()

# Вычисление степени разреженности
sparsity = 1 - (unique_pairs_count / (unique_items_count * events['user_id'].nunique()))

print("Степень разреженности U-I-матрицы: {:.2%}".format(sparsity))


Степень разреженности U-I-матрицы: 99.93%


In [15]:
# Подсчет общего количества ячеек в матрице
total_cells = events['user_id'].nunique() * events['item_id'].nunique()

# Подсчет количества пустых ячеек (ячеек без рейтинга)
empty_cells = total_cells - events.shape[0]

# Вычисление степени разреженности
sparsity = total_cells / empty_cells

print("Степень разреженности U-I-матрицы: {:.2f}".format(sparsity))

Степень разреженности U-I-матрицы: 1.00


## Реализация SVD-алгоритма

In [16]:
from surprise import Dataset, Reader
from surprise import SVD

# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set)

In [17]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set) 

In [18]:
from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae)

RMSE: 0.8289
MAE:  0.6474
0.8288711689059135 0.647437483750257


Проверка метрик на адекватность

In [19]:
from surprise import NormalPredictor

# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set)

In [20]:
rmse = accuracy.rmse(random_predictions)
mae = accuracy.mae(random_predictions)
print(rmse, mae)

RMSE: 1.2628
MAE:  1.0018
1.2628030301013033 1.0017726877569562


Задание. Удалите из events события для редких айтемов — таких, с которыми взаимодействовало менее N пользователей. Возьмите небольшое N, например 2–3 пользователя. Получите рекомендации, посчитайте метрики, оцените, как они изменились. Подумайте, с чем могут быть связаны такие изменения.

Попробовал мануал из https://habr.com/ru/companies/otus/articles/764222/

In [21]:
# Подсчет количества уникальных пользователей, взаимодействовавших с каждым айтемом
item_users_count = events.groupby('item_id')['user_id'].nunique()

# Удаление событий для айтемов, с которыми взаимодействовали менее N пользователей
N = 2
rare_items = item_users_count[item_users_count < N].index
events_filtered = events[~events['item_id'].isin(rare_items)]

In [22]:
from surprise.model_selection import train_test_split

# Создание объекта Reader для определения структуры данных
reader = Reader(rating_scale=(1, 5))

# Преобразование DataFrame в объект Dataset формата Surprise
data = Dataset.load_from_df(events_filtered[['user_id', 'item_id', 'rating']], reader)

# Разделение данных на обучающий и тестовый наборы
trainset, testset = train_test_split(data, test_size=0.2)

In [23]:
model = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02)
# Обучение модели на обучающем наборе
model.fit(trainset)

In [24]:
# Получение прогнозов модели на тестовом наборе
predictions = model.test(testset)

# Расчет MAE и RMSE
mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

MAE:  0.6477
RMSE: 0.8307
MAE: 0.6477165525975903
RMSE: 0.8307417913595345


## Получение рекомендаций

Создайте функцию, которая позволит получить рекомендации для конкретного пользователя, используя описанный подход. Дополните прекод.

In [25]:
def get_recommendations_svd(user_id, all_items, events, model, include_seen=True, n=5):

    """ возвращает n рекомендаций для user_id """
    
    # получим список идентификаторов всех книг
    all_items = set(events['item_id'].unique())
        
    # учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = set(events[events['user_id'] == user_id]['item_id'].unique())
        
        # книги, которые пользователь ещё не читал
        # только их и будем включать в рекомендации
        items_to_predict = list(all_items - seen_items)
    
    # получаем скоры для списка книг, т. е. рекомендации
    predictions = [model.predict(user_id, item_id) for item_id in items_to_predict]
    
    # сортируем рекомендации по убыванию скора и берём только n первых
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], columns=["item_id", "score"])

In [26]:
get_recommendations_svd(1296647, items, events_train, svd_model)

,item_id,score
0,24812,5.000000
1,7864312,4.981188
2,25793186,4.912001
3,12174312,4.898052
4,13208,4.894869


Если нужно вывести последние события для случайного пользователя и рекомендации для него. Это можно сделать с помощью функции

In [27]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)

user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations) 


user_id: 1345844
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
8,Orson Scott Card,"Ender's Shadow (Ender's Shadow, #1)",2012-05-30,2012-06-16,5,"{'Science Fiction': 3299, 'Fiction': 1329, 'Yo..."
9,E.L. James,"Fifty Shades of Grey (Fifty Shades, #1)",2012-05-04,2012-06-06,3,"{'Romance': 8712, 'Fiction': 3718, 'Adult Fict..."
10,Tina Fey,Bossypants,2012-04-08,2012-05-03,2,"{'Nonfiction': 8118, 'Autobiography-Memoir': 4..."
11,"Orson Scott Card, Stefan Rudnicki, Harlan Ellison","Ender's Game (Ender's Saga, #1)",2012-04-08,2012-05-21,5,"{'Science Fiction': 15315, 'Fiction': 7488, 'Y..."
12,Kristin Kimball,"The Dirty Life: On Farming, Food, and Love",2012-03-18,2012-04-05,4,"{'Nonfiction': 420, 'Autobiography-Memoir': 35..."
13,Jonathan Franzen,Freedom,2011-10-29,2011-11-17,4,"{'Fiction': 2920, 'Contemporary': 352, 'Novels..."
14,George R.R. Martin,"A Clash of Kings (A Song of Ice and Fire, #2)",2011-07-12,2011-08-04,5,"{'Fantasy': 31452, 'Fiction': 6543, 'Fantasy-E..."
15,George R.R. Martin,"A Game of Thrones (A Song of Ice and Fire, #1)",2011-06-24,2011-07-12,4,"{'Fantasy': 44086, 'Fiction': 10111, 'Fantasy-..."
16,Ally Condie,"Matched (Matched, #1)",2011-04-28,2011-05-15,4,"{'Young Adult': 7846, 'Science Fiction-Dystopi..."
17,J.K. Rowling,Harry Potter and the Deathly Hallows (Harry Po...,2010-11-28,2010-12-26,5,"{'Fantasy': 46667, 'Young Adult': 15403, 'Fict..."


Рекомендации


,item_id,score,author,title,genre_and_votes
0,22037424,5,"J.K. Rowling, Jonny Duddle, Tomislav Tomić",Harry Potter and the Prisoner of Azkaban (Harr...,"{'Fantasy': 49994, 'Young Adult': 15433, 'Fict..."
1,24812,5,Bill Watterson,The Complete Calvin and Hobbes,"{'Sequential Art-Comics': 867, 'Humor': 378, '..."
2,24814,5,Bill Watterson,It's a Magical World: A Calvin and Hobbes Coll...,"{'Sequential Art-Comics': 680, 'Humor': 381, '..."
3,40490,5,Arthur Bennett,The Valley of Vision: A Collection of Puritan ...,"{'Christian': 141, 'Prayer': 127, 'Religion-Th..."
4,2939264,5,يحيى بن شرف النووي,رياض الصالحين,"{'Religion': 52, 'Religion-Islam': 26, 'Nonfic..."


In [28]:
my_book = items[items['author'].str.contains('Tolkien', case=False, na=False)]
print(my_book[['item_id','title']])

          item_id                                              title
57870       77678  The Lays of Beleriand (The History of Middle-E...
236592    6784686                                     Leaf by Niggle
366252      15241         The Two Towers (The Lord of the Rings, #2)
480872    1335611                                      Tree and Leaf
521574     659469                          The Hobbit: Graphic Novel
542049      16907                             Smith of Wootton Major
629237       7329       Unfinished Tales of Númenor and Middle-Earth
805720     597790                              The Children of Húrin
806150       5907                                         The Hobbit
944968   32708664                                  Beren and Lúthien
1120947      2333                                Farmer Giles of Ham
1195424      7337                                 The Tolkien Reader
1195429      7332                                   The Silmarillion
1258628     23613  Sir Gawain and 

In [29]:
events.sample()

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month
4844060,1136829,29430012,2016-12-26,2017-01-22,True,4,False,2016-12-01


In [30]:
# Добавим нового пользователя с прочитанными мной книгами
new_user_id = events['user_id'].max() + 1
#new_user_id 

books_i_like = [2187513, 1258628]  # Книги, которые мне нравятся
new_user_events = pd.DataFrame({
    'user_id': [new_user_id] * len(books_i_like),
    'item_id': books_i_like,
    'started_at': ['2012-05-30', '2012-05-30'],  # Дата начала чтения
    'read_at': ['2012-06-16', '2012-06-16'],  # Дата окончания чтения
    'is_read': [True, True],
    'rating': [5, 4],
    'is_reviewed':[False, False],
    'started_at_month':['2012-05-30', '2012-05-30']
    
})

In [31]:
new_user_events

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month
0,1430585,2187513,2012-05-30,2012-06-16,True,5,False,2012-05-30
1,1430585,1258628,2012-05-30,2012-06-16,True,4,False,2012-05-30


In [34]:
pse_events = pd.concat([events_train, new_user_events], ignore_index=True)

In [35]:
get_recommendations_svd(1430585, items, pse_events, svd_model)

,item_id,score
0,24812,5.000000
1,11221285,4.914296
2,22037424,4.908423
3,33353628,4.872179
4,54741,4.872000


In [36]:
print("Рекомендации")
user_recommendations = get_recommendations_svd(1430585, items, pse_events, svd_model)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations) 

Рекомендации


,item_id,score,author,title,genre_and_votes
0,24812,5.000000,Bill Watterson,The Complete Calvin and Hobbes,"{'Sequential Art-Comics': 867, 'Humor': 378, '..."
1,11221285,4.914296,Brandon Sanderson,"The Way of Kings, Part 2 (The Stormlight Archi...","{'Fantasy': 641, 'Fiction': 46, 'Fantasy-Epic ..."
2,22037424,4.908423,"J.K. Rowling, Jonny Duddle, Tomislav Tomić",Harry Potter and the Prisoner of Azkaban (Harr...,"{'Fantasy': 49994, 'Young Adult': 15433, 'Fict..."
3,33353628,4.872179,Pénélope Bagieu,"Culottées #2 (Culottées, #2)","{'Sequential Art-Bande DessinÃ©e': 108, 'Femin..."
4,54741,4.872000,Quino,Toda Mafalda,"{'Sequential Art-Comics': 157, 'Humor': 47, 'S..."


# === Базовые подходы: коллаборативная фильтрация

Ниже приведён код для перекодировки идентификаторов

In [37]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])

/tmp/ipykernel_2091/1044897688.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_2091/1044897688.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_2091/1044897688.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [39]:
# Перекодируем идентификаторы пользователей
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train.loc[:, "user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test.loc[:, "user_id_enc"] = user_encoder.transform(events_test["user_id"])

# Перекодируем идентификаторы объектов
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train.loc[:, "item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test.loc[:, "item_id_enc"] = item_encoder.transform(events_test["item_id"])

In [40]:
events_train['item_id_enc'].max()

43304

In [41]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [42]:
import sys

sum([sys.getsizeof(i) for i in user_item_matrix_train.data])/1024**3

0.26370687410235405

In [43]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train) 

/home/mle-user/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mle-user/.local/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|███████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [02:55<00:00,  3.52s/it]


Описание прочих параметров и методов класса AlternatingLeastSquares можно посмотреть на странице технической документации на библиотеку implicit.
Чтобы получить рекомендации для пользователя с помощью модели ALS, используем такую функцию:

In [44]:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations

Используя get_recommendations_als, напишите код, который позволит для случайного пользователя просмотреть рекомендации в удобном формате: 
история с именами авторов и названием книг,
рекомендации с именами авторов и названием книг, seen-признаком (взаимодействовал ли уже пользователь с рекомендованной книгой).
Проанализируйте, релевантны ли рекомендации имеющейся истории.

In [45]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

user_id: 1418333


In [46]:
get_recommendations_als(user_item_matrix_train, als_model, user_id, user_encoder, item_encoder)

,item_id_enc,score,item_id
0,29392,1.294028,11235712
1,31273,1.149674,13206760
2,33783,1.127624,16096824
3,31274,1.040954,13206828
4,29910,1.026977,11870085


In [47]:
print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)

user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_als(user_item_matrix_train, als_model, user_id, user_encoder, item_encoder)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations) 

История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
126,"Natsuki Takaya, Alethea Nibley, Athena Nibley","Fruits Basket, Vol. 1",2012-06-30,2015-02-23,5,"{'Sequential Art-Manga': 5707, 'Fantasy': 485,..."
127,Alex Flinn,"Beastly (Beastly, #1; Kendra Chronicles, #1)",2012-01-10,2012-01-10,5,"{'Young Adult': 2672, 'Fantasy': 2570, 'Romanc..."
128,Becca Fitzpatrick,"Finale (Hush, Hush, #4)",2012-10-23,2012-10-24,5,"{'Young Adult': 1503, 'Fantasy': 1322, 'Parano..."
129,"Derek Ruiz, Becca Fitzpatrick, Jennyson Rosero...","Hush, Hush (Hush, Hush: The Graphic Novel, #1)",2014-06-02,2014-06-02,5,"{'Sequential Art-Graphic Novels': 107, 'Young ..."
130,Becca Fitzpatrick,"Silence (Hush, Hush, #3)",2012-09-21,2012-09-30,5,"{'Young Adult': 2116, 'Fantasy': 1769, 'Parano..."
131,Becca Fitzpatrick,"Hush, Hush (Hush, Hush, #1)",2012-09-07,2012-09-14,5,"{'Young Adult': 5341, 'Fantasy': 4355, 'Romanc..."
132,Becca Fitzpatrick,"Crescendo (Hush, Hush, #2)",2012-09-14,2012-09-21,5,"{'Young Adult': 2793, 'Fantasy': 2200, 'Parano..."
133,Simone Elkeles,"Chain Reaction (Perfect Chemistry, #3)",2012-07-26,2016-03-21,5,"{'Young Adult': 993, 'Romance': 993, 'Contempo..."
134,Simone Elkeles,"Rules of Attraction (Perfect Chemistry, #2)",2012-07-24,2016-03-19,5,"{'Young Adult': 1398, 'Romance': 1379, 'Contem..."
135,Simone Elkeles,"Perfect Chemistry (Perfect Chemistry, #1)",2012-07-23,2016-03-18,5,"{'Romance': 3197, 'Young Adult': 2980, 'Contem..."


Рекомендации


,item_id_enc,score,item_id,author,title,genre_and_votes
0,29392,1.294028,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)","{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
1,31273,1.149674,13206760,Marissa Meyer,"Scarlet (The Lunar Chronicles, #2)","{'Young Adult': 6930, 'Fantasy': 6267, 'Scienc..."
2,33783,1.127624,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
3,31274,1.040954,13206828,Marissa Meyer,"Cress (The Lunar Chronicles, #3)","{'Young Adult': 5813, 'Fantasy': 5413, 'Scienc..."
4,29910,1.026977,11870085,John Green,The Fault in Our Stars,"{'Young Adult': 20449, 'Fiction': 10338, 'Roma..."


Получим рекомендации для всех имеющихся пользователей. Выполните код ниже.

In [75]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, N=100)

Код возвращает рекомендации как список списков, это не очень удобно. Преобразуем его в более удобный формат — табличный.

In [77]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

In [78]:
als_recommendations.head()

,score,user_id,item_id
0,0.990941,1000000,3
1,0.896617,1000000,15881
2,0.864404,1000000,5
3,0.822254,1000000,6
4,0.774095,1000000,2


Сохраним полученные рекомендации в файл, они ещё нам пригодятся.

In [55]:
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("als_recommendations.parquet") 

Для удобства оценки добавим в датафрейм с рекомендациями истинные оценки из тестовой выборки:

In [89]:
events_test.query('item_id == 386372').head(1)

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month,user_id_enc,item_id_enc
466905,1356446,386372,2017-08-26,2017-09-13,True,5,False,2017-08-01,356446,12784


In [ ]:
als_recommendations.

In [86]:
bals_recommendations = (als_recommendations.merge(events_test[["user_id", "item_id", "rating"]], on=["user_id", "item_id"], how="left")) 

In [87]:
bals_recommendations.query('item_id == 386372')

,score,user_id,item_id,rating_test,rating
3161,0.070760,1000031,386372,NaN,NaN
6914,1.069344,1000069,386372,NaN,NaN
7966,0.024727,1000079,386372,NaN,NaN
8143,0.043379,1000081,386372,NaN,NaN
10855,0.002158,1000108,386372,NaN,NaN
...,...,...,...,...,...
43046155,0.017358,1430461,386372,NaN,NaN
43051943,0.004156,1430519,386372,NaN,NaN
43052391,0.039397,1430523,386372,NaN,NaN
43054227,0.043157,1430542,386372,NaN,NaN


In [79]:
als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "item_id", "rating"]]
               .rename(columns={"rating": "rating_test"}), 
           on=["user_id", "item_id"], how="left")
) 

In [81]:
als_recommendations.sample(5)

,score,user_id,item_id,rating_test
26577187,0.028890,1265771,386372,NaN
42584690,0.038998,1425846,3591262,NaN
14031076,0.127460,1140310,16160797,NaN
15311914,0.136014,1153119,3228917,NaN
5643860,0.054196,1056438,5326,NaN


In [84]:
als_recommendations.query('item_id == 386372')

,score,user_id,item_id,rating_test
3161,0.070760,1000031,386372,NaN
6914,1.069344,1000069,386372,NaN
7966,0.024727,1000079,386372,NaN
8143,0.043379,1000081,386372,NaN
10855,0.002158,1000108,386372,NaN
...,...,...,...,...
43046155,0.017358,1430461,386372,NaN
43051943,0.004156,1430519,386372,NaN
43052391,0.039397,1430523,386372,NaN
43054227,0.043157,1430542,386372,NaN


Подсчитать метрику NDCG для одного пользователя поможет готовая реализация из scikit-learn:

In [57]:
import sklearn.metrics

def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg

Умея считать NDCG для одного пользователя, посчитаем данную метрику, например, для 𝑘 =5 для всех пользователей из тестовой выборки. В результате каждому пользователю будет соответствовать одно значение NDCG@5. Запись “NDCG@5” означает, что метрика NDCG считается для пяти пользователей

In [58]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))

In [59]:
print(ndcg_at_5_scores.mean()) 

0.975946709792109


In [60]:
users_with_ndcg = ndcg_at_5_scores.dropna().shape[0]

# Общее количество пользователей
total_users = als_recommendations["user_id"].nunique()

# Доля пользователей, для которых удалось посчитать метрику NDCG
ratio_users_with_ndcg = users_with_ndcg / total_users
print(f"Доля пользователей с посчитанными значениями NDCG: {ratio_users_with_ndcg}")

Доля пользователей с посчитанными значениями NDCG: 0.04002926251495059


In [69]:
import random

# Выбор корректных индексов для нескольких айтемов
num_items_to_recommend = 10
item_ids_to_recommend = random.sample(range(len(item_encoder.classes_)), num_items_to_recommend)

# Получение рекомендаций похожих айтемов
similar_items_recommendations = als_model.similar_items(item_ids_to_recommend, N=10, filter_items=None, items=None)

# Преобразование результатов в DataFrame
similar_items_df = pd.DataFrame({
    "item_id_enc": item_ids_to_recommend,
    "similar_items": similar_items_recommendations[0].tolist(),
    "similarity_score": similar_items_recommendations[1].tolist()
})

# Вывод и анализ результатов
print(similar_items_df)

   item_id_enc                                      similar_items  \
0        23853  [23853, 26397, 22456, 27952, 28657, 27776, 313...   
1        38162  [38162, 39157, 39873, 40204, 38801, 40477, 402...   
2        14725  [14725, 15742, 27609, 30305, 37715, 36797, 391...   
3        42797  [42797, 42960, 41707, 41632, 42513, 42151, 426...   
4        37214  [37214, 38461, 38187, 38192, 38938, 38580, 382...   
5        14815  [14815, 16839, 6609, 31702, 21575, 10756, 7540...   
6        13621  [13621, 5604, 5605, 5606, 3108, 13999, 6976, 1...   
7         4694  [4694, 5927, 429, 1328, 5928, 426, 427, 8820, ...   
8         8680  [8680, 5470, 32815, 10928, 5469, 29791, 35509,...   
9        25468  [25468, 473, 2200, 25300, 20264, 2916, 17582, ...   

                                    similarity_score  
0  [0.9999998807907104, 0.9197226762771606, 0.911...  
1  [1.0, 0.998270571231842, 0.9944217801094055, 0...  
2  [0.9999998807907104, 0.8171135783195496, 0.816...  
3  [1.0, 0.821989715

# === Базовые подходы: контентные рекомендации

Составим список всех возможных жанров по всем айтемам. Потом по нему будем составлять вектор с весами по жанрам для каждой книги. Из любопытства составим список жанров с подсчётом количества голосов по каждому из них.
Сначала преобразуем значения в genre_and_votes из текстового представления в тип в Python:

In [90]:
items["genre_and_votes"] = items["genre_and_votes"].apply(eval)

In [91]:
items.head(2)

,item_id,author,title,description,genre_and_votes,num_pages,average_rating,ratings_count,text_reviews_count,publisher,publication_year,country_code,language_code,format,is_ebook,isbn,isbn13,genre_and_votes_dict,genre_and_votes_str,item_id_enc
3,6066819,Jennifer Weiner,Best Friends Forever,Addie Downs and Valerie Adler were eight when ...,"{'Womens Fiction-Chick Lit': 739, 'Fiction': 442}",368,3.49,51184,3282,Atria Books,2009,US,eng,Hardcover,False,0743294297,9780743294294,"{'Academic': None, 'Academic-Academia': None, ...","Womens Fiction-Chick Lit 739, Fiction 442",23133
6,378460,Michael Halberstam,The Wanting of Levine,,"{'Politics': 1, 'Humor': 1}",<NA>,4.38,12,4,Berkley Publishing Group,1979,US,,Paperback,False,0425040887,9780425040881,"{'Academic': None, 'Academic-Academia': None, ...","Politics 1user, Humor 1user",12687


Теперь составьте список жанров с долями голосов по ним в genres.  Дополните и выполните следующий код:

In [92]:
def get_genres(items):

    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    
    genres_counter = {}
    
    for k, v, in items.iterrows():
        genre_and_votes = v["genre_and_votes"]# ваш код здесь #
        if genre_and_votes is None or not isinstance(genre_and_votes, dict):
            continue
        for genre, votes in genre_and_votes.items():
            # увеличиваем счётчик жанров
            try:
                genres_counter[genre] += votes # ваш код здесь #
            except KeyError:
                genres_counter[genre] = 0

    genres = pd.Series(genres_counter, name="votes")
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"
    
    return genres
   
genres = get_genres(items)

In [93]:
genres["score"] = genres["votes"] / genres["votes"].sum()
genres.sort_values(by="score", ascending=False).head(10) 

,name,votes,score
genre_id,,,
25,Fantasy,6850060,0.149651
1,Fiction,6406256,0.139955
38,Classics,3414934,0.074605
18,Young Adult,3296951,0.072027
34,Romance,2422614,0.052926
5,Nonfiction,1737406,0.037957
16,Historical-Historical Fiction,1531205,0.033452
20,Mystery,1371196,0.029956
24,Science Fiction,1218917,0.026629


# === Базовые подходы: валидация

# === Двухстадийный подход: метрики

# === Двухстадийный подход: модель

# === Двухстадийный подход: построение признаков